In [6]:
import pandas as pd
import numpy as np
import math
import os
from datetime import datetime
import subprocess
import shutil

def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

gort_root_path = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in'
root_output_path = '/data/shared/src/STV/NEON_TEAK/allen/figs/veg_input'
plotloc_csv_filename = '/data/shared/src/STV/NEON_TEAK/allen/data/NEON_struct-plant/matching_shapefiles_tiffs.csv'
shapefile_path = "/data/shared/src/arojas/NEON/data/raw/spatial/baseplots/TEAK_baseplots_utm/TEAK_baseplots_utm.shp"
R_generated_csv = '/data/shared/src/STV/NEON_TEAK/allen/data/tree_locations.csv'
orth_tiff_folder_2023 = '/data/shared/rsdata/orthophotos/TEAK/NEON_images-camera-ortho-mosaic/NEON.D17.TEAK.DP3.30010.001.2023-07.basic.20240905T014815Z.PROVISIONAL'

TEAK_plotloc_unique_df = pd.read_csv(plotloc_csv_filename)

date_subfolder = get_date_subfolder()
gort_date_path = os.path.join(gort_root_path, date_subfolder)
date_output_path = os.path.join(root_output_path, date_subfolder)

# Ensure the full save path exists
os.makedirs(gort_date_path, exist_ok=True)

app_indv_100_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_full_100.csv"))
app_indv_400_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_full_400.csv"))

app_indv_100_df = app_indv_100_df.dropna(subset=['height']).sort_values(by='plotID_x')
app_indv_400_df = app_indv_400_df.dropna(subset=['height']).sort_values(by='plotID_x')

app_indv_R_df = pd.read_csv(R_generated_csv)

app_indv_100_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_100.csv"))
app_indv_400_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_400.csv"))

In [ ]:
def process_gort_inputs(app_indv_locations_df, app_indv_df, TEAK_plotloc_unique_df, save_path, plot_ident):
    # Rename column and sort values
    app_indv_df = app_indv_df.rename(columns={'plotID_x': 'plotID'})
    app_indv_df = app_indv_df.sort_values(by='plotID', ascending=True)
    drop_unknown_df = app_indv_df[app_indv_df['subplotID'].str.contains('unknown') == False]

    # Obtains nique plotIDs count
    unique_plotIDs = drop_unknown_df['plotID'].unique()
    # Declaration of gort_inputs table
    gort_inputs = pd.DataFrame({'plot_id': unique_plotIDs, 'Tree Density': np.nan})
    
    # Tree Density Calculation
    for plot_id in unique_plotIDs:
        # For debugging purposes, we keep track of the number of individual growthForms that have and do not have unknowns (in the subplot column)
        # And keeps track of the individuals that do or do not have a relative location entry
        with_unknowns_df = app_indv_df[app_indv_df['plotID'] == plot_id]
        without_unknowns_df = drop_unknown_df[drop_unknown_df['plotID'] == plot_id]
        locations_df = app_indv_locations_df[app_indv_locations_df['plotID_x'] == plot_id]
        subplot_unique_count = len(without_unknowns_df['subplotID'].unique())
        
        tree_density = 0
        
        if plot_ident == '100':
            tree_density = len(without_unknowns_df['subplotID']) / 400
        elif plot_ident == '400':
            tree_density = len(without_unknowns_df['subplotID']) / 800
            
        gort_inputs.loc[gort_inputs['plot_id'] == plot_id, 'Tree Density'] = tree_density

        # Debug statements
        print(f"Plot ID: {plot_id}")
        # print(f"Number of trees (with unknowns): {len(with_unknowns_df['subplotID'])}")
        print(f"Number of trees (without unknowns): {len(without_unknowns_df['subplotID'])}")
        # print(f"Number of trees (with locations): {len(locations_df['subplotID'])}")
        print(f"Number of unique subplots (without unknowns): {subplot_unique_count}")
        print(f"Normalized count: {tree_density}")
        print("------")

    gort_inputs = gort_inputs.sort_values(by='plotID', ascending=True)

    noshrub_noheightna_df = app_indv_df[app_indv_df['height'].notna()]
    noshrub_noheightna_df = noshrub_noheightna_df.copy()

    # Used two different methods to initialize column values to np.nan
    if 'Vertical_Crown_Radius' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df['Vertical_Crown_Radius'] = np.nan
    if 'Crown_Center_Height' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df.loc[:, 'Crown_Center_Height'] = np.nan

    # Mean Vertical Crown Radius Calculation
    for index, row in noshrub_noheightna_df.iterrows():
        vert_crown_radius = (row['height'] - row['baseCrownHeight']) / 2

        noshrub_noheightna_df.at[index, 'Vertical_Crown_Radius'] = vert_crown_radius
        noshrub_noheightna_df.at[index, 'Crown_Center_Height'] = row['height'] - vert_crown_radius

    mean_vert_crown_radius = noshrub_noheightna_df.groupby('plotID')['Vertical_Crown_Radius'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_vert_crown_radius, on='plotID', how='left')
    gort_inputs.rename(columns={'Vertical_Crown_Radius': 'Mean_Vert_Crown_Radius'}, inplace=True)

    # Mean/STD Crown Center Height Calculation
    mean_std_crown_center_height = noshrub_noheightna_df.groupby('plotID')['Crown_Center_Height'].agg(['mean', 'std']).reset_index()
    mean_std_crown_center_height.rename(columns={'mean': 'Mean_Crown_Center_Height', 'std': 'Std_Crown_Center_Height'}, inplace=True)
    gort_inputs = pd.merge(gort_inputs, mean_std_crown_center_height, on='plotID', how='left')
    
    # Mean Horizontal Crown Radius Calculation
    mean_crown_radius = noshrub_noheightna_df.groupby('plotID')['crown_radius'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_crown_radius, on='plotID', how='left')
    gort_inputs.rename(columns={'crown_radius': 'Mean_Crown_Radius'}, inplace=True)

    # Individual and Total Crown Volume Calculation
    if 'Crown_Volume' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df['Crown_Volume'] = np.nan

    for index, row in noshrub_noheightna_df.iterrows():
        crown_volume = (4/3) * math.pi * (row['crown_radius']**2) * row['Vertical_Crown_Radius']
        noshrub_noheightna_df.at[index, 'Crown_Volume'] = crown_volume

    total_crown_volume = noshrub_noheightna_df.groupby('plotID')['Crown_Volume'].sum().reset_index()
    gort_inputs = pd.merge(gort_inputs, total_crown_volume, on='plotID', how='left')
    gort_inputs.rename(columns={'Crown_Volume': 'Total Crown_Volume'}, inplace=True)

    # Mean Crown Volume Calculation
    gort_inputs['Mean_Crown_Volume'] = (4/3) * math.pi * (gort_inputs['Mean_Crown_Radius'] ** 2) * gort_inputs['Mean_Vert_Crown_radius']
        
    
    gort_inputs.at[plot_id, 'Fa'] = avg_crown_volume

    for index1, row1 in TEAK_plotloc_unique_df.iterrows():
        named_location = row1['namedLocation']
        lai_average = row1['LAI_Average_exclude_negatives']
        plot_area = row1['plot_area']
        
        

        for index2, row2 in gort_inputs.iterrows():
            plot_id = row2['plotID']
            total_crown_vol_per_plot = row2['Total Crown_Volume']
            if plot_id == named_location:
                gort_inputs.at[index2, 'Fa'] = (lai_average / total_crown_vol_per_plot) * plot_area
                break

    if 'h1' not in gort_inputs.columns:
        gort_inputs['h1'] = np.nan
    if 'h2' not in gort_inputs.columns:
        gort_inputs['h2'] = np.nan

    for index, row in gort_inputs.iterrows():
        mean_crct_height = row['Mean_Crown_Center_Height']
        std_crct_height = row['Std_Crown_Center_Height']
        h1 = mean_crct_height - (2 * std_crct_height)
        h2 = mean_crct_height + (2 * std_crct_height)

        if h1 < row['Mean_Vert_Crown_Radius']:
            h1 = row['Mean_Vert_Crown_Radius']

        gort_inputs.at[index, 'h1'] = h1
        gort_inputs.at[index, 'h2'] = h2

    # Mean Height and Mean Horizontal Crown Radius Calculation + dz values
    mean_height = noshrub_noheightna_df.groupby('plotID')['height'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_height, on='plotID', how='left')
    gort_inputs.rename(columns={'height': 'Mean_Height'}, inplace=True)

    mean_horz_crown_radius = noshrub_noheightna_df.groupby('plotID')['crown_radius'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_horz_crown_radius, on='plotID', how='left')
    gort_inputs.rename(columns={'crown_radius': 'Mean_Horz_Crown_Radius'}, inplace=True)

    gort_inputs['dz'] = 0.1
    gort_inputs = gort_inputs.round(4)

    gort_inputs = gort_inputs[gort_inputs['h1'].notna()]
    filename = "gort_input_values_" + plot_ident + ".csv"
    gort_inputs.to_csv(os.path.join(save_path, filename), index=False)

    return gort_inputs

def output_gort_files(gort_inputs, save_path, gort_ident):
    gort_save_path = os.path.join(save_path, gort_ident)
    for index, row in gort_inputs.iterrows():
        output_file_name = os.path.join(gort_save_path, f"{row['plotID']}.in")

        os.makedirs(gort_save_path, exist_ok=True)

        with open(output_file_name, 'w') as file:
            file.write('1\n')
            file.write('\t' + str(row['Tree Density']))
            file.write('\t' + str(row['Fa']))
            file.write('\t' + str(row['h1']))
            file.write('\t' + str(row['h2']))
            file.write('\t' + str(row['dz']))
            file.write('\t' + str(row['Mean_Horz_Crown_Radius']))
            file.write('\t' + str(row['Mean_Vert_Crown_Radius']))
            file.write('\t' + '0.00')
            file.write('\t' + '0.4291')
            file.write('\t' + '0.4239')
            file.write('\t' + '0.35156')
            file.write('\t' + '1')
        
        print(f"file saved to:  {output_file_name}")

In [7]:
print("100m2: \n")
gort_inputs_100_df = process_gort_inputs(app_indv_100_locations_df, app_indv_100_df, TEAK_plotloc_unique_df, date_output_path, "100")
print("400m2: \n")
gort_inputs_400_df = process_gort_inputs(app_indv_400_locations_df, app_indv_400_df, TEAK_plotloc_unique_df, date_output_path, "400")

100m2: 

Plot ID: TEAK_001
Number of trees (without unknowns): 11
Number of unique subplots (without unknowns): 1
Normalized count: 0.0275
------
Plot ID: TEAK_002
Number of trees (without unknowns): 14
Number of unique subplots (without unknowns): 4
Normalized count: 0.035
------
Plot ID: TEAK_003
Number of trees (without unknowns): 17
Number of unique subplots (without unknowns): 3
Normalized count: 0.0425
------
Plot ID: TEAK_005
Number of trees (without unknowns): 10
Number of unique subplots (without unknowns): 2
Normalized count: 0.025
------
Plot ID: TEAK_006
Number of trees (without unknowns): 12
Number of unique subplots (without unknowns): 3
Normalized count: 0.03
------
Plot ID: TEAK_007
Number of trees (without unknowns): 12
Number of unique subplots (without unknowns): 4
Normalized count: 0.03
------
Plot ID: TEAK_010
Number of trees (without unknowns): 4
Number of unique subplots (without unknowns): 3
Normalized count: 0.01
------
Plot ID: TEAK_011
Number of trees (withou

In [8]:
output_gort_files(gort_inputs_100_df, gort_date_path, "Gort_100")
output_gort_files(gort_inputs_400_df, gort_date_path, "Gort_400")

file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_001.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_002.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_003.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_005.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_006.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_007.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_010.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_011.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_012.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_013.in
file saved to:  /dat

In [9]:
def run_gort_teak(input_dir, output_dir, process_dir):
    date_subfolder = get_date_subfolder()

    # Find all .in files recursively in the input directory
    fplot = [os.path.join(root, file)
             for root, _, files in os.walk(input_dir)
             for file in files if file.endswith('.in')]
    
    # Get the current working directory at the start of the script
    initial_dir = os.getcwd()

    # Process each .in file
    for infile in fplot:
        # Get the relative path of the infile with respect to the input directory
        rel_path = os.path.relpath(infile, input_dir)
        
        # Determine the corresponding output file path
        out_file = os.path.join(output_dir, rel_path).replace('.in', '.out')
        
        # Ensure the directory for the output file exists
        os.makedirs(os.path.dirname(out_file), exist_ok=True)
        
        # Copy the input file to gort.in in the process directory
        process_infile = os.path.join(process_dir, 'gort.in')
        shutil.copy(infile, process_infile)
        print(f"Copied {infile} to {process_infile}")
        
        # Change to the process directory
        os.chdir(process_dir)
        
        # Run the Fortran script in the process directory
        cmd_run = './gort_lidar > temp.txt'
        result = subprocess.run(cmd_run, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Error running GORT model: {result.stderr}")
            break
        
        # Move gort.out from the process directory to the corresponding output file path
        process_outfile = os.path.join(process_dir, 'gort.out')
        shutil.move(process_outfile, out_file)
        print(f'Moved {process_outfile} to {out_file}')
        
        # Change back to the initial directory
        os.chdir(initial_dir)

In [10]:
gort_input_dir = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/' + date_subfolder
gort_output_dir = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/' + date_subfolder
gort_process_dir = '/data/shared/src/STV/simscenes/LidarSimulation/model/'

run_gort_teak(gort_input_dir, gort_output_dir, gort_process_dir)

Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_001.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_09/Gort_100/TEAK_001.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_002.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_09/Gort_100/TEAK_002.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100/TEAK_003.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_09/Gort_100/TEAK_003.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_09/Gort_100